In [1]:
import pandas as pd
import os
import tqdm
from sklearn.preprocessing import OneHotEncoder
import utils.featurecreations as featurecreations
import importlib
import torch
import torch.nn as nn

In [2]:
dfs = []
for file in tqdm.tqdm(os.listdir('data')):
    try:
        df = pd.read_excel(f'data/{file}')
        dfs.append(df)
    except:
        print(file,'failed')

100%|██████████| 8/8 [15:16<00:00, 114.52s/it]

~$2021_Sep01_to_Dec31.xlsx failed


In [54]:
data = pd.concat(dfs)

In [4]:
data = data.reset_index(drop=True)

In [5]:
data.to_feather('all_races_data.fth')

In [55]:
df = pd.read_feather('all_races_data.fth')

In [56]:
data = df

In [57]:
data['meetingId'] = data['meetingId'].astype(int).astype(str)
data['raceId'] = data['raceId'].astype(int).astype(str)
data['horseId'] = data['horseId'].astype(int).astype(str)

In [58]:
data.columns

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting', 'isBarrierTrial',
       'hasSectionals', 'resulted', 'raceNo', 'raceName', 'prizeMoney',
       'starters', 'startTime', 'class', 'distance', 'prizeMoneyBreakDown',
       'ageRestrictions', 'SexRestrictions', 'jockeyRestrictions',
       'trackCondition', 'trackConditionNumber', 'raceTime', 'sectional',
       'sectionalDistance', 'PFRaceTime', 'PFRaceL600Time', 'limitWeight',
       'weightType', 'jumps', 'groupStatus', 'tabNo', 'position', 'margin',
       'horse', 'weight', 'barrier', 'inRun', 'flucs', 'priceSP', 'stewards',
       'priceBF', 'gears', 'sex', 'age', 'CareerStarts', 'sire', 'dam',
       'foalDate', 'trainer', 'jockey', 'jockeyId', 'trainerId', 'jock_claim',
       'Reliable', 'RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R',
       'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP',
       'RunStyle', 'Publish

In [59]:
race1 = data[data['raceId']=='879736']

In [60]:
race1

,meetingId,raceId,horseId,meetingDate,track,trackAbbrev,State,MPC,railPosition,TABMeeting,...,Pos600,Pos200,Marg600,Marg200,MargFin,Last600Time,Last200Time,ABML600,ABML200,ABMFin
1167,178543,879736,921720,03/09/2021,Albury,ALB,NSW,C,True,True,...,5,3,5.1,1.3,0.0,35.20,12.50,-3.5,0.8,5.4
1168,178543,879736,909170,03/09/2021,Albury,ALB,NSW,C,True,True,...,6,4,5.3,2.5,3.7,35.84,12.96,0.7,3.8,9.8
1169,178543,879736,941305,03/09/2021,Albury,ALB,NSW,C,True,True,...,4,2,3.6,1.1,0.5,35.52,12.60,-1.1,1.7,6.0
1170,178543,879736,884221,03/09/2021,Albury,ALB,NSW,C,True,True,...,3,5,2.5,3.4,6.7,36.86,13.36,7.6,6.3,13.4
1171,178543,879736,917728,03/09/2021,Albury,ALB,NSW,C,True,True,...,2,6,2.4,4.3,11.5,37.68,14.00,13.5,11.0,19.2
1172,178543,879736,893930,03/09/2021,Albury,ALB,NSW,C,True,True,...,1,1,0.0,0.0,3.5,36.68,13.36,6.8,6.6,9.6


In [61]:
# data['isBarrierTrial'].value_counts()

In [62]:
# data['stats_sum'] = data[stats_cols].fillna(0).sum(axis='columns' )

In [63]:
# data['stats_sum']

In [64]:
# data['stats_sum'].value_counts()

In [65]:
stats_cols = ['RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R',
       'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP',
       'ClsDiff','ETRPrRk','ETRPrice'
       ]

In [66]:
stats = data[stats_cols]

In [67]:
data.RunStyle.head()

0      op/mf
1      op/mf
2    no data
3      mf/bm
4      mf/bm
Name: RunStyle, dtype: object

In [68]:
data.RunStyle.value_counts()

RunStyle
mf         105388
bm         102396
op          72467
op/mf       67420
mf/bm       64239
l           31433
no data     28342
Name: count, dtype: int64

In [69]:
run_style_ohe = OneHotEncoder(sparse_output=False)
run_style_one_hot = run_style_ohe.fit_transform(data[['RunStyle']])
track_ohe = OneHotEncoder(sparse_output=False)
track_one_hot = track_ohe.fit_transform(data[['track']])

In [70]:
data["run_style_one_hot"] = run_style_one_hot.tolist()
data["tracks_one_hot"] = track_one_hot.tolist()

In [71]:
len(data["tracks_one_hot"].iloc[0])

404

In [72]:
horse_form = data.groupby(['horseId'], sort=False, as_index=False)

In [73]:
importlib.reload(featurecreations)

<module 'utils.featurecreations' from 'c:\\Users\\Nick\\Documents\\GitHub\\BetfairSpringCarnival\\src\\utils\\featurecreations.py'>

In [74]:
data = featurecreations.generate_prev_race(data, horse_form)

In [75]:
data['prev_race'].iloc[-1]

'1028124'

In [76]:
data['dog_id'] = data['horseId']
data['box'] = data['barrier']
data['bfSP'] = data['priceBF']
data['margin'] = data['MargFin']
data['StartPrice'] = data['priceBF']
data['place'] = data['position']
data['runtime'] = data['ITimeToFin']
data['date'] = pd.to_datetime(data['meetingDate'], format='mixed', dayfirst=True ).dt.date
# data['date'] = pd.Timestamp(data['date'])
data['state'] = data['State']
data['dog_name'] = data['horse']
# data = data[]

In [77]:
# data.barrier?
data.columns
data['State']

0          NZ
1          NZ
2          NZ
3          NZ
4          NZ
         ... 
566456    NSW
566457    NSW
566458    NSW
566459    NSW
566460    NSW
Name: State, Length: 566461, dtype: object

In [78]:
form_only_stats = data[stats_cols].fillna(-1.0)
stats = pd.Series(form_only_stats.values.tolist())

In [79]:
# data = data.drop(columns=stats_cols)
data['stats'] = stats

In [80]:
data = data.query('barrier > 0 and barrier <=20')
data.barrier.value_counts()

barrier
2     63006
1     62999
3     62922
4     62345
5     60379
6     56501
7     50654
8     43384
9     35025
10    26898
11    18467
12    12453
13     6134
14     3657
15      933
16      509
17       99
18       67
19       13
20        9
Name: count, dtype: int64

In [81]:
data['box'] = data['barrier']

In [82]:
data['runtime'] = data['ITimeToFin']
data['race_grade'] = data['class']

In [83]:
data['stats_cols'] = str([stats_cols])
data['stats_cols'] = data['stats_cols'].astype('category')

In [84]:
data = data[~data['isBarrierTrial']]

In [85]:
data = data.reset_index(drop=True)

In [86]:
data.shape

(473112, 108)

In [87]:
race = data[data['raceId']=='879552']

In [88]:
race.barrier

504    7
505    5
506    6
507    2
508    1
509    8
510    4
511    3
Name: barrier, dtype: int64

In [89]:
empty_margin_list = [100]*20
margin_list = [x for x in race["margin"]]
boxes_list = [int(x) for x in race['box']]
for n,x in enumerate(boxes_list):
    empty_margin_list[x-1] = margin_list[n]

In [90]:
print(empty_margin_list)

[0.0, 1.9, 0.2, 1.3, 4.8, 3.9, 1.8, 0.1, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


In [91]:
def boosted_softmin(input):
    sft_min = nn.Softmin(dim=-1)
    return sft_min(torch.exp(input))

In [92]:
adjustedMargin = (boosted_softmin(torch.tensor(empty_margin_list)))

In [93]:
adjustedMargin

tensor([3.5963e-01, 1.2204e-03, 2.8820e-01, 2.4922e-02, 0.0000e+00, 3.4271e-22,
        2.3058e-03, 3.2372e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00])

In [94]:
adjustedMargin.to('cuda:0')

tensor([3.5963e-01, 1.2204e-03, 2.8820e-01, 2.4922e-02, 0.0000e+00, 3.4271e-22,
        2.3058e-03, 3.2372e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0')

In [95]:
data.reset_index(drop=True).to_feather('horsey_racing_v1.fth')

In [96]:
data = data[data['raceId']=='891328']

In [97]:
print(data[['place','barrier']])

       place  barrier
80515      8        1
80516     14        6
80517     12        8
80518      6        2
80519     13        9
80520      1       12
80521      2        4
80522      4        5
80523     10       14
80524      3        3
80525      7       10
80526      9       11
80527      5       13
80528     11        7


In [98]:
print(data.shape)
# data = data.

(14, 108)


In [99]:
data.columns

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting',
       ...
       'bfSP', 'StartPrice', 'place', 'runtime', 'date', 'state', 'dog_name',
       'stats', 'race_grade', 'stats_cols'],
      dtype='object', length=108)

In [100]:
x = data.groupby('track')['barrier'].max()

In [101]:
data.barrier.value_counts()

barrier
1     1
6     1
8     1
2     1
9     1
12    1
4     1
5     1
14    1
3     1
10    1
11    1
13    1
7     1
Name: count, dtype: int64

In [102]:
data.groupby('raceId').count().mean()

meetingId      14.0
horseId        14.0
meetingDate    14.0
track          14.0
trackAbbrev    14.0
               ... 
state          14.0
dog_name       14.0
stats          14.0
race_grade     14.0
stats_cols     14.0
Length: 107, dtype: float64